In [2]:
class RSAReceiver :
    def __init__(self, bit_len: int):
        """
        Arguments :
        bit_len : A lower bound for the number of bits of N,
        the second argument of the public and secret key .
        """
        self.largo = bit_len

    def get_public_key(self) -> bytearray:
        """
        Returns :
        public_key : Public key expressed as a Python ’bytearray ’ using the
        PEM format . This means the public key is divided in:
        (1) The number of bytes of e (4 bytes )
        (2) the number e (as many bytes as indicated in (1))
        (3) The number of bytes of N (4 bytes )
        (4) the number N (as many bytes as indicated in (3))
        """
    def decrypt(self, ciphertext: bytearray) -> str:
        """
        Arguments :
        ciphertext : The ciphertext to decrypt
        Returns :
        message : The original message
        """

In [3]:
class RSASender:
    def __init__(self, public_key: bytearray):
        """
        Arguments :
        public_key : The public key that will be used to encrypt messages
        """
        self.public_key = public_key
    
    def encrypt(self, message: str) -> bytearray :
        """
        Arguments :
        message : The plaintext message to encrypt
        Returns :
        ciphertext : The encrypted message
        """
